In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/MyDrive/데이터분석_프로젝트/ai_factory/data_set")
os.getcwd()

'/content/drive/MyDrive/데이터분석_프로젝트/ai_factory/data_set'

In [ ]:
import random
import numpy as np

import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [4]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")
sub_df = pd.read_csv("answer_sample.csv")

In [5]:
# monitor vibe 보정
type_1 = train_df[train_df["type"] == 1]
outlier_index = type_1[type_1["motor_vibe"] > 15].index
outlier = train_df.iloc[outlier_index]["motor_temp"] / 20
outlier = outlier.values
train_df.loc[outlier_index, "motor_vibe"] = outlier

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [38]:
from sklearn.preprocessing import StandardScaler

# Preprocess the data
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_df)
test_data_scaled = scaler.transform(test_df)

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

In [57]:
input_dim = train_data_scaled.shape[1]
encoding_dim = 64
hidden_dim = 32
hidden_dim_2 = 16

In [69]:
class Encoder(nn.Module):
    def __init__(self, input_dim, encoding_dim, hidden_dim,hidden_dim_2):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, encoding_dim, bias = False)
        self.fc1 = nn.Linear(input_dim, encoding_dim)
        self.fc2 = nn.Linear(encoding_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim_2)
        #self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

class Decoder(nn.Module):
    def __init__(self, input_dim, encoding_dim, hidden_dim, hidden_dim_2):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(hidden_dim_2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, encoding_dim)
        self.fc3 = nn.Linear(encoding_dim, input_dim)
        #self.fc4 = nn.Linear(encoding_dim, input_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

encoder = Encoder(input_dim, encoding_dim, hidden_dim, hidden_dim_2)
decoder = Decoder(input_dim, encoding_dim, hidden_dim, hidden_dim_2)
autoencoder = nn.Sequential(encoder, decoder)

In [70]:
print(autoencoder)

Sequential(
  (0): Encoder(
    (fc1): Linear(in_features=8, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=32, bias=True)
    (fc3): Linear(in_features=32, out_features=16, bias=True)
  )
  (1): Decoder(
    (fc1): Linear(in_features=16, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=8, bias=True)
  )
)


In [71]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters())

In [72]:
# Train the autoencoder with early stopping
epochs = 500
batch_size = 8
early_stopping_patience = 5
train_data_scaled_tensor = torch.tensor(train_data_scaled, dtype=torch.float32)
train_data_loader = torch.utils.data.DataLoader(train_data_scaled_tensor, batch_size=batch_size, shuffle=True)

min_val_loss = float('inf')
patience_count = 0
for epoch in range(epochs):
    running_loss = 0.0
    for data in train_data_loader:
        optimizer.zero_grad()
        outputs = autoencoder(data)
        loss = criterion(outputs, data)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    val_loss = criterion(autoencoder(torch.tensor(test_data_scaled, dtype=torch.float32)), torch.tensor(test_data_scaled, dtype=torch.float32)).item()
    print('epoch=', epoch, 'val_loss= ', val_loss)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        patience_count = 0
    else:
        patience_count += 1
    if patience_count == early_stopping_patience:
        print(f"Early stopping at epoch {epoch}")
        break

epoch= 0 val_loss=  0.008215351030230522
epoch= 1 val_loss=  0.0020310382824391127
epoch= 2 val_loss=  0.0019126670667901635
epoch= 3 val_loss=  0.002040114253759384
epoch= 4 val_loss=  0.0017711416585370898
epoch= 5 val_loss=  0.0015200100606307387
epoch= 6 val_loss=  0.0011312243295833468
epoch= 7 val_loss=  0.0014178184792399406
epoch= 8 val_loss=  0.000946094689425081
epoch= 9 val_loss=  0.0009155287407338619
epoch= 10 val_loss=  0.000873592565767467
epoch= 11 val_loss=  0.0008925119764171541
epoch= 12 val_loss=  0.001142720109783113
epoch= 13 val_loss=  0.0009627453982830048
epoch= 14 val_loss=  0.0008207615464925766
epoch= 15 val_loss=  0.001287951716221869
epoch= 16 val_loss=  0.0007382490439340472
epoch= 17 val_loss=  0.0006329798488877714
epoch= 18 val_loss=  0.0006325542344711721
epoch= 19 val_loss=  0.001466982183046639
epoch= 20 val_loss=  0.0005138388369232416
epoch= 21 val_loss=  0.0005037974915467203
epoch= 22 val_loss=  0.0008681064355187118
epoch= 23 val_loss=  0.00052

In [ ]:
print(autoencoder)

In [73]:
# Calculate the reconstruction error for test_data
test_data_reconstructed = autoencoder(torch.tensor(test_data_scaled, dtype=torch.float32)).detach().numpy()
reconstruction_error = ((test_data_scaled - test_data_reconstructed) ** 2).mean(axis=1)

# Set a threshold for considering a test point as an anomaly
threshold = np.percentile(reconstruction_error, 95.35)

# Predict anomalies in test_data
test_predictions_basic_epoch500_pt5 = (reconstruction_error > threshold).astype(int)

In [74]:
reconstruction_error

array([2.60749513e-05, 2.54783596e-05, 3.19446157e-05, ...,
       5.99233072e-05, 6.05534790e-05, 6.18688776e-05])

In [75]:
from collections import Counter

# Count the number of 0s and 1s in y_pred_binary
count = Counter(test_predictions_basic_epoch500_pt5)
num_0 = count[0]
num_1 = count[1]

print("Number of 0s: ", num_0)
print("Number of 1s: ", num_1)

Number of 0s:  7045
Number of 1s:  344


In [76]:
a = test_predictions_basic_epoch500_pt5.tolist()

In [78]:
sub_df["label"] = a

In [81]:
sub_df.to_csv("sub_df_1.csv", index = False)